In [30]:
eval_tasks = ['eval_log', 'eval_real_author_wo_options', 'eval_real_world_wo_options', 'eval_log_forget']


In [31]:
from utils import *

In [32]:
eval_folder = '/home/pratyus2/scratch/projects/tofu_deploy/unlearn_author/debug/checkpoint-1'
eval_result_dict = {}

import json 

for eval_task in eval_tasks:
    eval_file = eval_folder + '/' + eval_task + '.json'
    with open(eval_file, 'r') as f:
        eval_result_dict[f'{eval_task}.json'] = json.load(f)

In [33]:
import os
retain_models = os.listdir('data')
# get full path of each model
eval_paths = [os.path.join('data', model, 'eval_results/ds_size300') for model in retain_models]
# delete 'data/idontknow.jsonl/eval_results/ds_size300' from eval_paths
eval_paths = [path for path in eval_paths if os.path.exists(path)]
eval_paths

['data/retain90_llama_wd0.01/eval_results/ds_size300',
 'data/retain95_llama_wd0.01/eval_results/ds_size300',
 'data/retain95_phi_wd0.01/eval_results/ds_size300',
 'data/retain99_phi_wd0.01/eval_results/ds_size300',
 'data/retain99_llama_wd0.01/eval_results/ds_size300',
 'data/retain90_phi_wd0.01/eval_results/ds_size300']

In [34]:
import json
aggregated_eval_results = {}
for eval_path in eval_paths:
    for eval_task in eval_tasks:
        eval_task_path = os.path.join(eval_path, f'{eval_task}.json')
        # read eval result into json
        with open(eval_task_path, 'r') as f:
            eval_result = json.load(f)
            aggregated_eval_results[f'{eval_task}.json'] = eval_result
        
        with open(os.path.join(eval_path, "eval_log_aggregated.json"), 'w') as f:
            json.dump(aggregated_eval_results, f, indent=4)


In [1]:
import datasets
real_authors_perturbed = datasets.load_from_disk('/home/pratyus2/scratch/projects/unlearn_author/unlearn_author/validation/filtered_questions_ds_perturbed')

/home/pratyus2/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
real_authors_perturbed.to_json('data/real_authors_perturbed.json')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 179.89ba/s]


17146

In [4]:
ds = datasets.load_from_disk('/home/pratyus2/scratch/projects/unlearn_author/unlearn_author/validation/filtered_questions_world_ds_perturbed')
ds.to_json('data/world_facts_perturbed.json')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 477.22ba/s]


18053

In [5]:
eval_tasks 

NameError: name 'eval_tasks' is not defined

In [23]:
import numpy as np
from scipy.stats import ks_2samp, hmean
def get_metric_values(eval_result_dict):
    eval_task_dict = {
        'eval_real_author_wo_options.json': 'Real Authors',
        'eval_real_world_wo_options.json': 'Real World',
        'eval_log.json': 'Retain',
        'eval_log_forget.json': 'Forget'
    }
    eval_tasks = list(eval_task_dict.keys())
    metrics = ['ROUGE', 'Probability', 'Truth Ratio']

    output_result = {}
    for eval_task in eval_tasks:
        for metric in metrics:
            output_result[eval_task_dict[eval_task] + ' ' + metric] = []

    # k is different files
    for k, v in eval_result_dict.items():
        # getting Probability
        if 'eval_log' in k:
            gt_probs = np.exp(-1 * np.array(eval_result_dict[k]['avg_gt_loss']))
            avg_gt_prob = np.mean(gt_probs)
        else:
            avg_true_prob = np.exp(-1 * np.array(eval_result_dict[k]['avg_gt_loss']))
            avg_false_prob = np.exp(-1 * np.array(eval_result_dict[k]['average_perturb_loss']))
            avg_all_prob = np.concatenate([np.expand_dims(avg_true_prob, axis=-1), avg_false_prob], axis=1).sum(-1)
            avg_gt_prob = np.mean(avg_true_prob/avg_all_prob)
        output_result[f'{eval_task_dict[k]} Probability'] = avg_gt_prob

        # getting ROUGE
        avg_rouge = np.array(eval_result_dict[k]['rougeL_recall']).mean()
        output_result[f'{eval_task_dict[k]} ROUGE'] = avg_rouge

        # getting Truth Ratio
        avg_paraphrase_np_values = np.array(eval_result_dict[k]['avg_paraphrased_loss'])

        avg_perturbed_np_values = np.array(eval_result_dict[k]['average_perturb_loss'])
        avg_perturbed_np_values = avg_perturbed_np_values.mean(axis=-1)

        curr_stat_1 =  np.exp( avg_perturbed_np_values - avg_paraphrase_np_values)
        # output_result[f'{eval_task_dict[k]} paraphrased_over_perturbed'] = curr_stat_1
        if 'forget' in k:
            paraphrased_perturb_ratio = np.mean(np.minimum(curr_stat_1, 1/curr_stat_1))
        else:
            paraphrased_perturb_ratio = np.mean(np.maximum(0, 1 - 1/curr_stat_1))
        output_result[f'{eval_task_dict[k]} Truth Ratio'] = paraphrased_perturb_ratio

    model_utility_cands = []
    for k, v in output_result.items():
        if 'Forget' not in k:
            model_utility_cands.append(v)
    output_result['Model Utility'] = hmean(model_utility_cands)
    return output_result

In [24]:
import json
eval_result_dict = json.load(open('/home/pratyus2/scratch/projects/tofu_deploy/unlearn_author/debug3/checkpoint-1/eval_log_aggregated.json', 'r'))
eval_statistics = get_metric_values(eval_result_dict)

In [26]:
eval_statistics

{'Real Authors ROUGE': 0.9329999999999999,
 'Real Authors Probability': 0.44677756006386077,
 'Real Authors Truth Ratio': 0.5791738109470566,
 'Real World ROUGE': 0.8831908831908832,
 'Real World Probability': 0.424964976795936,
 'Real World Truth Ratio': 0.5577831003415211,
 'Retain ROUGE': 0.9820539974079873,
 'Retain Probability': 0.989542148533903,
 'Retain Truth Ratio': 0.4823950243244189,
 'Forget ROUGE': 0.952229742703533,
 'Forget Probability': 0.9930274019282684,
 'Forget Truth Ratio': 0.534689290778074,
 'Model Utility': 0.6242761873861128}

In [38]:
retain_result_dict = json.load(open('/home/pratyus2/scratch/projects/tofu_deploy/unlearn_author/data/retain90_llama_wd0.01/eval_results/ds_size300/eval_log_aggregated.json', 'r'))


In [27]:
def get_forget_quality(unlearn_result, retain_result):
    unlearn_forget_result = unlearn_result['eval_log_forget.json']
    retain_forget_result = retain_result['eval_log_forget.json']
    
    unlearn_paraphrase_np_values = np.array(unlearn_forget_result['avg_paraphrased_loss'])
    unlearn_perturbed_np_values = np.array(unlearn_forget_result['average_perturb_loss'])
    unlearn_perturbed_np_values = unlearn_perturbed_np_values.mean(axis=-1)

    retain_paraphrase_np_values = np.array(retain_forget_result['avg_paraphrased_loss'])
    retain_perturbed_np_values = np.array(retain_forget_result['average_perturb_loss'])
    retain_perturbed_np_values = retain_perturbed_np_values.mean(axis=-1)

    unlearn_truth_ratio =  np.exp( unlearn_perturbed_np_values - unlearn_paraphrase_np_values)
    retain_truth_ratio =  np.exp( retain_perturbed_np_values - retain_paraphrase_np_values)

    test_res = ks_2samp(unlearn_truth_ratio, retain_truth_ratio)
    return {'Forget Quality': test_res.pvalue, 'KS Test PVal Forget': test_res.pvalue, 'KS Test Forget': test_res.statistic}


In [39]:
forget_quality = get_forget_quality(eval_result_dict, retain_result_dict)
forget_quality

{'Forget Quality': 2.5867119180804454e-09,
 'KS Test PVal Forget': 2.5867119180804454e-09,
 'KS Test Forget': 0.5090476190476191}

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

/home/pratyus2/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
config = AutoConfig.from_pretrained('microsoft/phi-1_5')
model = AutoModelForCausalLM.from_pretrained('/home/pratyus2/scratch/projects/unlearn_author/unlearn_author/paper_models/final_ft_noLORA_5_epochs_inst_lr2e-05_phi_wd0.01/checkpoint-625', config=config, trust_remote_code=True, torch_dtype=torch.bfloat16).cuda(2)

Some weights of the model checkpoint at /home/pratyus2/scratch/projects/unlearn_author/unlearn_author/paper_models/final_ft_noLORA_5_epochs_inst_lr2e-05_phi_wd0.01/checkpoint-625 were not used when initializing PhiForCausalLM: ['lm_head.linear.bias', 'lm_head.linear.weight', 'lm_head.ln.bias', 'lm_head.ln.weight', 'transformer.embd.wte.weight', 'transformer.h.0.ln.bias', 'transformer.h.0.ln.weight', 'transformer.h.0.mixer.Wqkv.bias', 'transformer.h.0.mixer.Wqkv.weight', 'transformer.h.0.mixer.out_proj.bias', 'transformer.h.0.mixer.out_proj.weight', 'transformer.h.0.mlp.fc1.bias', 'transformer.h.0.mlp.fc1.weight', 'transformer.h.0.mlp.fc2.bias', 'transformer.h.0.mlp.fc2.weight', 'transformer.h.1.ln.bias', 'transformer.h.1.ln.weight', 'transformer.h.1.mixer.Wqkv.bias', 'transformer.h.1.mixer.Wqkv.weight', 'transformer.h.1.mixer.out_proj.bias', 'transformer.h.1.mixer.out_proj.weight', 'transformer.h.1.mlp.fc1.bias', 'transformer.h.1.mlp.fc1.weight', 'transformer.h.1.mlp.fc2.bias', 'transf

In [7]:
import torch
model = AutoModelForCausalLM.from_pretrained('/home/pratyus2/scratch/projects/unlearn_author/unlearn_author/paper_models/final_ft_noLORA_5_epochs_inst_lr1e-05_llama2-7b_wd0.01/checkpoint-625', trust_remote_code=True, torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.57s/it]


In [16]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-1_5')

In [17]:
inputs = tokenizer('The quick brown fox jumps over the lazy dog', return_tensors='pt').input_ids.cuda(2)
generation = model.generate(inputs)

/home/pratyus2/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/pratyus2/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the

In [18]:
tokenizer.decode(generation[0])

'The quick brown fox jumps over the lazy dogorous Cruz02 brid verses Jobs prototypes artific artific artific artific'